In [1]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import string

import nltk 
from nltk import pos_tag
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import movie_reviews, stopwords,wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from rake_nltk import Rake

import numpy as np
from numpy import arange,atleast_2d,argsort

from collections import defaultdict

from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from ast import literal_eval

In [2]:
helpfulCutOffRatings = 20
reviewDataPath = 'amazon/vsmall.csv'
metadataPath = 'amazon/vsmall_metadata.json'
topN = 5

In [3]:
def removeUnusedReviewInfo(rdata):
    rdata = rdata.drop(['unixReviewTime','reviewTime','reviewerName'],axis=1)
    return rdata
    
def createReviewerInfo(data):
    reviewerInfo = data[['reviewerID', 'reviewerName']]
    return reviewerInfo

In [4]:
def createUserMovieMatrix(udata,mdata):
    fullData = udata.merge(mdata, on='asin',how='outer')
    userMovieMatrix = fullData.pivot_table(index='reviewerID', columns = 'asin',values = 'overall').reset_index().rename_axis(None, axis=1)
    userMovieMatching = pd.crosstab(index = fullData['reviewerID'], columns=df['asin'],dropna = False)
    userMovieMatrix = userMovieMatrix.replace(np.nan,0)
    userMovieMatrix = userMovieMatrix.drop(['reviewerID'],axis = 1)
    return userMovieMatrix, userMovieMatching


In [5]:
# Classify data into negative or positive
def classifyBaseReviewRating(x):
    if x > 2.5:
        return "positive"
    if x == 2.5:
        return "neutral"
    if x < 2.5:
        return "negative"

In [6]:
def cleanData(r):
    r = str(r)
    if len(r) == 0:
        return ''
    if type(r) == float:
        return ''
    return r

In [49]:
def getWordnetType(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def cleanReview(r):
    r = str(r)
    if len(r) == 0:
        return ''
    if type(r) == float:
        return ''
    r = r.lower()
    #remove puncutation
    r = [eachWord.strip(string.punctuation) for eachWord in r.split(" ") ]
    #remove numbers
    r = [eachWord for eachWord in r if not any(l.isdigit() for l in eachWord)]
    
    #remove stop words
    stop = stopwords.words('english')
    r = [w for w in r if w not in stop]
    
    #remove words which are empty
    r = [w for w in r if len(w) >0]
    
    #add pos_tags
    pos_tags = pos_tag(r)
    
    #word lemmatize
    r = [WordNetLemmatizer().lemmatize(t[0], getWordnetType(t[1])) for t in pos_tags]
    
    #remove useless words
    r = [w for w in r if len(w) > 1]
    
    #recreate review
    r = " ".join(r)
    return r

# get compound score from VADER and append it as review rating
def getSentimentScore(rdata):
    sa = SentimentIntensityAnalyzer()
    rdata["sentiments"] = rdata["reviewText"].apply(lambda x: sa.polarity_scores(x))
    rdata = pd.concat([rdata.drop(['sentiments'], axis=1), rdata['sentiments'].apply(pd.Series)], axis=1)
    rdata['helpful'] = rdata['helpful'].apply(lambda x: getHelpfulScore(x))
    #compound score from VADER is from -1 to +1. Scaling it to 0 to 5 and multiplying it with helpfulness score
    rdata['reviewRating'] = (1+(rdata['compound']+1)*2)
#     rdata['reviewRating'] = (1+(rdata['compound']+1)*2) * rdata['helpful']
    rdata = rdata.drop(['compound','pos','neg','neu'],axis = 1)
    rdata = rdata.drop(['reviewText','helpful'],axis=1)
    return rdata
    

In [8]:
def getHelpfulScore(x):
    if x == None:
        return 1.0
    x = literal_eval(x)
    a = float(x[0])
    b = float(x[1])

    if b != 0 and a + b > helpfulCutOffRatings-1:
        return a / (a + b)
    else:
        return 1.0

In [9]:
def generateSentimentScoreOnReviews(rdata):
    reviewerInfo = createReviewerInfo(rdata)
    rdata = removeUnusedReviewInfo(rdata)
    rdata["reviewType"] = rdata["overall"].apply(classifyBaseReviewRating)
    rdata['reviewText'] = rdata['reviewText'] + rdata['summary']
    rdata = rdata.drop(['summary'],axis = 1)
    rdata['reviewText'] = rdata['reviewText'].apply(cleanData)    
    rdata['reviewText'] = rdata['reviewText'].apply(lambda x:cleanReview(x))
    rdata = getSentimentScore(rdata)
    return rdata


In [10]:
def getMovieData(path):
    return pd.read_json(path,lines=True,typ='frame')

def preProcessMovieData(moviedata):
    moviedata.sort_values("asin",inplace=True)
    moviedata.drop_duplicates(subset='asin',keep=False,inplace = True)
    moviedata = moviedata.reset_index(drop=True)
    moviedata = moviedata.drop(['rank','main_cat','image','also_buy','also_view','price','details','feature','date','tech1'],axis = 1,errors='ignore')
    return moviedata



In [11]:
def getCosineSimilarity(moviedata):
    count = CountVectorizer()
    count_matrix = count.fit_transform(moviedata['cleanText'])
    # generating the cosine similarity matrix
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    return cosine_sim

def getMovieSimilarityFromPlot(moviedata):
    # initializing the new column
    moviedata['impWords'] = ''
    moviedata['cleanText'] = ''
    for index, movie in moviedata.iterrows():
        tot = ''
        description = movie['description']
        if description != None:
            if type(description) != float:
                if len(description) != 0:
                    for i in description:
                        if type(i) != str:
                            print(description)
                        tot = tot + i

        #remove "Movies & TV", "Genre for Featured Categories", "Movies", "Independently Distributed", "All Titles", "All", "4-for-3 DVD", 
        ignoreWords = ["Movies & TV", "Genre for Featured Categories", "Movies", "Independently Distributed", "All Titles", "All", "4-for-3 DVD"]
        category = movie['category']
        if category != None:
            if type(category) != float:
                for w in category:
                    if w not in ignoreWords:
                        tot = tot+w
        brand = movie['brand']
        if brand != None:
            if type(brand) != float:
                tot = tot + i
        r = Rake()
        r.extract_keywords_from_text(tot)
        # getting the dictionary whith key words as keys and their scores as values
        key_words_dict_scores = r.get_word_degrees()

        # assigning the key words to the new column for the corresponding movie
        movie['impWords'] = list(key_words_dict_scores.keys())
        movie['cleanText'] = " ".join(map(str,movie['impWords']))
    moviedata = moviedata.drop(['category','brand','description','impWords'],axis = 1)
    cosine_sim =  getCosineSimilarity(moviedata)
    moviedata = moviedata.drop(['cleanText'],axis = 1)
    return cosine_sim, moviedata


In [12]:
def getMovieAvgRating(moviedata,rdataf):
    movieAvgRating = rdataf.groupby('asin')['overall'].agg(['mean', 'median', 'size'])
    movieAvgRating.columns = ['rating_mean', 'rating_median', 'num_ratings']
    return movieAvgRating


In [13]:
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls


In [16]:
def getPredictionFromMovieContent(moviedata, rdata, testSplit):
    moviedata = preProcessMovieData(moviedata)
    # craete cosine similarity matrix
    cosine_sim, moviedata = getMovieSimilarityFromPlot(moviedata)
    rdata, testdata = train_test_split(rdata, test_size=testSplit)
#     userMovieMatrix = createUserMovieMatrix(rdata,moviedata)
    rdata = rdata.merge(moviedata, on='asin',how='left')
    movieAvgRating = getMovieAvgRating(moviedata,rdata)
    movieAvgRating.columns = ['rating_mean', 'rating_median', 'num_ratings']
    moviedata = moviedata.merge(movieAvgRating, on='asin',how='left')
    
    predictions = getUserRatingPredictionFromMovieContent(cosine_sim, rdata, testdata, moviedata)
    return predictions

In [17]:
def getUserRatingPredictionFromMovieContent(cosine_sim, rdata, testdata, moviedata):
    userMovieMatrix,userMovieMatching = createUserMovieMatrix(rdata, moviedata)
    sumUserMovieWeight = np.dot(userMovieMatrix, cosine_sim)
    sumSimFoRM2M = np.dot(userMovieMatching, cosine_sim)
    userMoviePredict = sumUserMovieWeight/sumSimFoRM2M
    userMovieMatching = 1 - userMovieMatching
    userMoviePredictMissingMovies = userMoviePredict * userMovieMatching
    userMoviePredictMissingMovies = userMoviePredictMissingMovies.reset_index()
    userMoviePredictMissingMovies = userMoviePredictMissingMovies.replace(0,np.nan)
    userMoviePredictMissingMovies = pd.melt(userMoviePredictMissingMovies, id_vars='reviewerID', 
                                        value_vars=list(finRM.columns[1:]),
                                        var_name='movie', 
                                        value_name='rating')
    userMoviePredictMissingMovies = userMoviePredictMissingMovies[userMoviePredictMissingMovies['Rating'].notna()]
    userMoviePredictMissingMovies = userMoviePredictMissingMovies.reset_index(drop=True)
    userMoviePredictMissingMovies[key] = tuple(zip(userMoviePredictMissingMovies.reviewerID, 
                                                  userMoviePredictMissingMovies.movie))
    predictions = pd.Series(userMoviePredictMissingMovies['rating'].values,index=df.key).to_dict()
    return predictions

In [ ]:
finRM
finRM = finRM.replace(0,np.nan)

finRM = pd.melt(finRM, id_vars='reviewerID', 
            value_vars=list(finRM.columns[1:]), # list of days of the week
            var_name='movie', 
            value_name='Rating')
df = finRM
df = df[df['Rating'].notna()]
df = df.reset_index(drop=True)
df['c'] = tuple(zip(df.reviewerID,df.movie))
pd.Series(df['Rating'].values,index=df.c).to_dict()

In [324]:
rsmall = pd.read_csv('vvsmall.csv', names=['reviewerID','asin','overall'], skiprows=1, sep=' ')
meta = pd.read_json('vvsmall_medata.json',lines=True,typ='frame')
meta
rsmall
df = rsmall.merge(meta, on='asin',how='outer')
df

df = rsmall.merge(meta, on='asin',how='outer')
df.replace(np.nan,0)
# user index 
dfBin = pd.crosstab(index=df['reviewerID'], columns=df['asin'],dropna = False)
dfBin
# df.pivot_table(index='reviewerID', columns = 'asin',values = 'overall').reset_index()
tt = df.pivot_table( index='reviewerID',columns = 'asin',values = 'overall',dropna=False).reset_index().rename_axis(None, axis=1)
tt
tt = tt.replace(np.nan,0)
tt = tt.drop(['reviewerID'],axis = 1)
tt
cosTest = np.array([[1, 0.5, 0.25,0.25], [0.5, 1, 0.4,0.3],[0.25,0.4,1,0.6],[0.25,0.3,0.6,1]])
m2mCosim = pd.DataFrame(cosTest)
m2mCosim
mul = np.dot(tt,m2mCosim)
binMul = np.dot(dfBin,m2mCosim)
binMul

finR = mul/binMul
finR


array([[3.66666667, 3.71428571, 3.28571429, 3.33333333],
       [5.        , 5.        , 5.        , 5.        ]])

In [327]:
dfBin = 1-dfBin

In [385]:
# dfBin


In [503]:
finRM = dfBin * finR

In [504]:
finRM = finRM.reset_index()

In [505]:
finRM
finRM = finRM.replace(0,np.nan)

finRM = pd.melt(finRM, id_vars='reviewerID', 
            value_vars=list(finRM.columns[1:]), # list of days of the week
            var_name='movie', 
            value_name='Rating')
df = finRM
df = df[df['Rating'].notna()]
df = df.reset_index(drop=True)
df['c'] = tuple(zip(df.reviewerID,df.movie))
pd.Series(df['Rating'].values,index=df.c).to_dict()

{('u1', 'm1'): 3.6666666666666665,
 ('u2', 'm2'): 5.0,
 ('u2', 'm3'): 5.0,
 ('u1', 'm4'): 3.3333333333333335,
 ('u2', 'm4'): 5.0}

In [398]:
# ttt= finRM.drop(['reviewerID'],axis=1)
# ttt

asin,m1,m2,m3,m4
0,3.666667,0.0,0.0,3.333333
1,0.000000,5.0,5.0,5.000000


In [445]:
# finRM.loc[0,'reviewerID']

In [444]:
# c = finRM.columns
# list(c)[1:]

In [443]:
# finRM.to_dict()

In [447]:
# finRM.columns()

In [469]:
# finRM.set_index('reviewerID',inplace=True)
finRM = finRM.replace(0,np.nan)

finRM = pd.melt(finRM, id_vars='reviewerID', 
            value_vars=list(finRM.columns[1:]), # list of days of the week
            var_name='movie', 
            value_name='Rating')
df = finRM
df = df[df['Rating'].notna()]
df

,reviewerID,movie,Rating
0,u1,m1,3.666667
3,u2,m2,5.000000
5,u2,m3,5.000000
6,u1,m4,3.333333
7,u2,m4,5.000000


In [470]:
df = df.reset_index(drop=True)

In [494]:
df['c'] = tuple(zip(df.reviewerID,df.movie))


In [496]:
pd.Series(df['Rating'].values,index=df.c).to_dict()

{('u1', 'm1'): 3.6666666666666665,
 ('u2', 'm2'): 5.0,
 ('u2', 'm3'): 5.0,
 ('u1', 'm4'): 3.3333333333333335,
 ('u2', 'm4'): 5.0}

In [487]:
zip((df.reviewerID, df.movie),df.Rating)

In [402]:
# finRM.loc[i,'reviewerID']
for i,j in enumerate(ttt.columns):
    print(finRM.loc[i,'reviewerID'])
    print(i,j)

0 m1
1 m2
2 m3
3 m4


In [403]:
for i in finRM.items():
    print(i)

('reviewerID', 0    u1
1    u2
Name: reviewerID, dtype: object)
('m1', 0    3.666667
1    0.000000
Name: m1, dtype: float64)
('m2', 0    0.0
1    5.0
Name: m2, dtype: float64)
('m3', 0    0.0
1    5.0
Name: m3, dtype: float64)
('m4', 0    3.333333
1    5.000000
Name: m4, dtype: float64)


In [350]:
u = finRM['reviewerID']

In [365]:
ut = pd.DataFrame(u)
ut

,reviewerID
0,u1
1,u2


In [346]:
finRM = finRM.drop(['reviewerID'],axis=1)

In [366]:
mt = pd.DataFrame(finRM.T)
mt

,0,1
asin,,
m1,3.666667,0.0
m2,0.000000,5.0
m3,0.000000,5.0
m4,3.333333,5.0


In [374]:
# pd.merge(ut, mt, left_index=True, right_index=True)
pd.merge(ut, mt, left_index=True, right_index=True)
# pd.merge(mt,ut,on='')


,reviewerID,0,1


In [ ]:
# m2mCosim = pd.DataFrame(cosine_sim)
# indexWithMovieId = moviedata['asin']
# m2mCosim.columns = [str(indexWithMovieId[int(col)]) for col in m2mCosim.columns]
# m2mCosim.index = [indexWithMovieId[idx] for idx in m2mCosim.index]
# # m2mCosim.iloc[0].sort_values(ascending=False)[:10]
# userRatings = rdataf[rdataf.reviewerID == 'A1GHUN5HXMHZ89']
# userRatings.head(2)
# userDataWithScore = moviedata.reset_index().merge(userRatings, on='asin')

# userDataWithScore['weight'] = userDataWithScore['overall']/5
# # df_user_data_with_tags['weight'].values
# # df_user_data_with_tags
# user_profile = np.dot(count_matrix[userDataWithScore['index'].values].toarray().T, userDataWithScore['weight'].values)
# C = cosine_similarity(atleast_2d(user_profile), count_matrix)
# R = argsort(C)[:, ::-1]
# recommendations = [i for i in R[0] if i not in userDataWithScore['index'].values]
# rdataf['asin'][recommendations].head(10)


In [21]:
# Based on field in and type
def getSurpriseRatingPrediction(data, ratingCol, testSplit, algo, userBased):
    ratings_dict = {'itemID': list(data.asin),
                'userID': list(data.reviewerID),
                'rating': list(data[ratingCol])}
    df = pd.DataFrame(ratings_dict)
    
    # A reader is still needed but only the rating_scale param is required.
    # The Reader class is used to parse a file containing ratings.
    reader = Reader(rating_scale=(0.5, 5.0))
    # The columns must correspond to user id, item id and ratings (in that order).
    data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=testSplit)
    if algo == 'svd':
        algo = SVD()
    elif algo == 'knn':
        sim_options = {'name': 'cosine',
               'user_based': userBased
               }
        algo = algo = KNNBaseline(sim_options=sim_options)
    algo.fit(trainset)
    print('mean',trainset.global_mean)
    predictions = algo.test(testset)
    return predictions
    

In [29]:
def getPredictions(rdata, moviedata, testSplit, topN):
    # get SVD predictions
    svdPredictions = getSurpriseRatingPrediction(rdata, 'overall', testSplit, 'svd', False )
    
    #get Knn user based predictions based on ratings
    userBasedKNNPredictions = getSurpriseRatingPrediction(rdata, 'overall', testSplit, 'knn', True)
    
    #get Knn movie based predictions based on ratings
    ratingBasedKNNPredictions = getSurpriseRatingPrediction(rdata, 'overall', testSplit, 'knn', False)
    
    #get Knn movie based predictions based on reviews
    rdata = generateSentimentScoreOnReviews(rdata)
    reviewBasedKNNPredictions = getSurpriseRatingPrediction(rdata, 'reviewRating', testSplit, 'knn', False)
    
    #get content based predictions
    contentBasedPredictions = getPredictionFromMovieContent(moviedata, rdata, testSplit)
    
    topNRec = getFinalRec(svdPredictions, userBasedKNNPredictions,
                                           ratingBasedKNNPredictions, reviewBasedKNNPredictions,
                                          contentBasedPredictions, topN)
    return topNRec
    
    

In [30]:
# Combine all the predictions score by weighted average
def combinePredictions(svdPredictions, userBasedKNNPredictions,
                                           ratingBasedKNNPredictions, reviewBasedKNNPredictions,
                                          contentBasedPredictions):
    svdPredictionsNew = defaultdict(list)
    for uid, iid, true_r, est, _ in svdPredictions:
            svdPredictionsNew[uid].append((iid, est))
    
    
    userBasedKNNPredictionsNew = defaultdict(list)
    for uid, iid, true_r, est, _ in userBasedKNNPredictions:
            userBasedKNNPredictionsNew[uid].append((iid, est))
            
    
    ratingBasedKNNPredictionsNew = defaultdict(list)
    for uid, iid, true_r, est, _ in ratingBasedKNNPredictions:
            ratingBasedKNNPredictionsNew[uid].append((iid, est))
    
    
    reviewBasedKNNPredictionsNew = defaultdict(list)
    for uid, iid, true_r, est, _ in reviewBasedKNNPredictions:
            reviewBasedKNNPredictionsNew[uid].append((iid, est))
    
    # contentBasedPredictions is already of the form like above
    # TODO: add scores of different methods and recommend
    
    return top_recs
    

In [31]:
def getFinalRec(svdPredictions, userBasedKNNPredictions,
                                           ratingBasedKNNPredictions, reviewBasedKNNPredictions,
                                          contentBasedPredictions, topN):
    
    predictions = combinePredictions(svdPredictions, userBasedKNNPredictions,
                                           ratingBasedKNNPredictions, reviewBasedKNNPredictions,
                                          contentBasedPredictions)
    
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]we
    return top_recs
    

,reviewerID,asin,overall,title
0,u1,m2,4.0,m2
1,u2,m1,5.0,m1
2,u1,m3,3.0,m3
3,0,m4,0.0,m4


,reviewerID,asin,overall
0,u1,m2,4
1,u2,m1,5
2,u1,m3,3


,reviewerID,asin,overall,title
0,u1,m2,4.0,m2
1,u2,m1,5.0,m1
2,u1,m3,3.0,m3
3,0,m4,0.0,m4


,reviewerID,asin,overall,title
0,u1,m2,4.0,m2
1,u2,m1,5.0,m1
2,u1,m3,3.0,m3
3,NaN,m4,NaN,m4


asin,m1,m2,m3,m4
reviewerID,,,,
u1,0,1,1,0
u2,1,0,0,0


array([[3.66666667, 3.71428571, 3.28571429, 3.33333333],
       [5.        , 5.        , 5.        , 5.        ]])

,reviewerID,m1,m2,m3,m4
0,u1,0.0,4.0,3.0,0.0
1,u2,5.0,0.0,0.0,0.0


,m1,m2,m3,m4
0,0.0,4.0,3.0,0.0
1,5.0,0.0,0.0,0.0


asin,m1,m2,m3,m4
reviewerID,,,,
u1,0,1,1,0
u2,1,0,0,0


,0,1,2,3
0,1.00,0.5,0.25,0.25
1,0.50,1.0,0.40,0.30
2,0.25,0.4,1.00,0.60
3,0.25,0.3,0.60,1.00


array([[0.75, 1.4 , 1.4 , 0.9 ],
       [1.  , 0.5 , 0.25, 0.25]])

array([[3.66666667, 3.71428571, 3.28571429, 3.33333333],
       [5.        , 5.        , 5.        , 5.        ]])

In [ ]:
'''
movies = pd.read_csv('movies.csv')
movies_dict = {'itemID': list(movies.movieId),
                'title': list(movies.title)}
df2 = pd.DataFrame(movies_dict)
'''
# svd
rmse = []
size = []

In [ ]:
# read data as sample pandas dataframe
rdata = pd.read_csv('mvr_test_small.tsv', names=['reviewerID','asin','reviewerName','helpful','reviewText','overall','summary','unixReviewTime','reviewTime'], skiprows=1, sep='\t')
rdata.reset_index(inplace=True)
rdata = rdata.drop(['index'],axis =1)
moviedata = pd.read_json('meta_Movies_and_TV.json',lines=True,typ='frame')
for i in arange(0.05, 1, 0.05):
    getPredictions(rdata, moviedata, i, topN)
    size.append(i)
    rmse.append(accuracy.rmse(predictions, verbose=True))
    top3_recommendations = get_top3_recommendations(predictions)
    #print('top recommendations for SVD', top3_recommendations)
    
    rmse1.append(accuracy.rmse(predictions1, verbose=True))
    top3_recommendations = get_top3_recommendations(predictions1)
    #print('top recommendations using Item-Item sim', top3_recommendations)
    
    rmse2.append(accuracy.rmse(predictions2, verbose=True))
    top3_recommendations = get_top3_recommendations(predictions2)
    #print('top recommendations using User-User sim', top3_recommendations)
    
    #for uid, user_ratings in top3_recommendations.items():
        #print(uid, [df2.loc[df2['itemID'] == iid, 'title'].iloc[0] for (iid, _) in user_ratings])
        #print(uid, [result['itemID'] == iid for (iid, _) in user_ratings])
        #print(uid, [result.loc[result['itemID'] == iid, 'title'].iloc[0] for (iid, _) in user_ratings])
    #precisions, recalls = precision_recall_at_k(predictions, k=3, threshold=4)

    # Precision and recall can then be averaged over all users
    #print(sum(prec for prec in precisions.values()) / len(precisions))
    #print(sum(rec for rec in recalls.values()) / len(recalls))

In [ ]:

plt.style.use('seaborn-whitegrid')
fig = plt.figure()
ax = plt.axes()
plt.plot(size, rmse)
plt.show()

In [301]:
rsmall = pd.read_csv('vvsmall.csv', names=['reviewerID','asin','overall'], skiprows=1, sep=' ')
meta = pd.read_json('vvsmall_medata.json',lines=True,typ='frame')
meta
rsmall
df = rsmall.merge(meta, on='asin',how='outer')
df
df = rsmall.merge(meta, on='asin',how='outer')
df.replace(np.nan,0)
# user index 
dfBin = pd.crosstab(index=df['reviewerID'], columns=df['asin'],dropna = False)
# df.pivot_table(index='reviewerID', columns = 'asin',values = 'overall').reset_index()
tt = df.pivot_table( index='reviewerID',columns = 'asin',values = 'overall',dropna=False).reset_index().rename_axis(None, axis=1)
tt = tt.replace(np.nan,0)
tt
tt = tt.drop(['reviewerID'],axis = 1)
tt
dfBin
cosTest = np.array([[1, 0.5, 0.25,0.25], [0.5, 1, 0.4,0.3],[0.25,0.4,1,0.6],[0.25,0.3,0.6,1]])
m2mCosim = pd.DataFrame(cosTest)
m2mCosim
mul = np.dot(tt,m2mCosim)
binMul = np.dot(dfBin,m2mCosim)
binMul
f = mul/binMul

In [320]:
dfBin

asin,m1,m2,m3,m4
reviewerID,,,,
u1,0,1,1,0
u2,1,0,0,0


In [308]:
mul

array([[2.75, 5.2 , 4.6 , 3.  ],
       [5.  , 2.5 , 1.25, 1.25]])

In [309]:
m2mCosim

,0,1,2,3
0,1.00,0.5,0.25,0.25
1,0.50,1.0,0.40,0.30
2,0.25,0.4,1.00,0.60
3,0.25,0.3,0.60,1.00


In [310]:
binMul

array([[0.75, 1.4 , 1.4 , 0.9 ],
       [1.  , 0.5 , 0.25, 0.25]])

In [311]:
f

array([[3.66666667, 3.71428571, 3.28571429, 3.33333333],
       [5.        , 5.        , 5.        , 5.        ]])

In [317]:
pd.DataFrame(f)

,0,1,2,3
0,3.666667,3.714286,3.285714,3.333333
1,5.000000,5.000000,5.000000,5.000000


In [322]:
dfBin = 1- dfBin

asin,m1,m2,m3,m4
reviewerID,,,,
u1,1,0,0,1
u2,0,1,1,1


In [ ]:
# m2mCosim = pd.DataFrame(cosine_sim)
# indexWithMovieId = moviedata['asin']
# m2mCosim.columns = [str(indexWithMovieId[int(col)]) for col in m2mCosim.columns]
# m2mCosim.index = [indexWithMovieId[idx] for idx in m2mCosim.index]
# # m2mCosim.iloc[0].sort_values(ascending=False)[:10]
# userRatings = rdataf[rdataf.reviewerID == 'A1GHUN5HXMHZ89']
# userRatings.head(2)
# userDataWithScore = moviedata.reset_index().merge(userRatings, on='asin')

# userDataWithScore['weight'] = userDataWithScore['overall']/5
# # df_user_data_with_tags['weight'].values
# # df_user_data_with_tags
# user_profile = np.dot(count_matrix[userDataWithScore['index'].values].toarray().T, userDataWithScore['weight'].values)
# C = cosine_similarity(atleast_2d(user_profile), count_matrix)
# R = argsort(C)[:, ::-1]
# recommendations = [i for i in R[0] if i not in userDataWithScore['index'].values]
# rdataf['asin'][recommendations].head(10)
